In [1]:
import xarray as xr
import glob
import intake
import numpy as np
import seaborn as sns
from scipy import stats
import os

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
import xesmf as xe




In [3]:
from plotting_maps.acs_plotting_maps import plot_acs_hazard_multi, plot_acs_hazard, plot_data, cmap_dict, regions_dict
from matplotlib import colors, cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from dask.diagnostics import ProgressBar
# ProgressBar().register()

import dask
from dask.distributed import Client
client = Client(threads_per_worker=3, n_workers=14)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 42,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40653,Workers: 14
Dashboard: /proxy/8787/status,Total threads: 42
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:46499,Total threads: 3
Dashboard: /proxy/41515/status,Memory: 8.94 GiB
Nanny: tcp://127.0.0.1:33209,


2025-09-23 08:46:32,317 - distributed.nanny - WARNING - Restarting worker
2025-09-23 08:49:16,719 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle bbd6df577083e6328c634460d3125a13 initialized by task ('rechunk-merge-rechunk-transfer-53f0ab7fbcca6a4eab89f4bac94768ea', 0, 0, 5, 1, 0, 5) executed on worker tcp://127.0.0.1:39935
2025-09-23 08:49:17,079 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f64cc547bdf8860cd521d2f65b29392f initialized by task ('rechunk-merge-rechunk-transfer-53f0ab7fbcca6a4eab89f4bac94768ea', 0, 1, 3, 1, 1, 3) executed on worker tcp://127.0.0.1:34751
2025-09-23 08:49:17,252 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 0fa8005d18b05583e9cbbe22f00c7fa0 initialized by task ('rechunk-merge-rechunk-transfer-53f0ab7fbcca6a4eab89f4bac94768ea', 0, 0, 3, 1, 0, 3) executed on worker tcp://127.0.0.1:33447
2025-09-23 08:49:17,379 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 9c5f75b9ab06a9983ff8cee6ea3d844b initialized by ta

In [4]:
# for BARRA

In [5]:
# BARRA process has to be a bit different because the files are saved in single time slices, compared to single level with whole year


In [6]:
ds_target = xr.open_dataset("/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_ACCESS-CM2_historical_r4i1p1f1_BOM_BARPA-R_v1-r1_6hr_1979.nc")
ds_target


<xarray.Dataset> Size: 4GB
Dimensions:    (time: 1460, lat: 436, lon: 777)
Coordinates:
  * time       (time) datetime64[ns] 12kB 1979-01-01 ... 1979-12-31T18:00:00
  * lon        (lon) float64 6kB 88.04 88.19 88.34 88.5 ... 207.6 207.8 207.9
  * lat        (lat) float64 3kB -53.58 -53.42 -53.27 ... 13.32 13.48 13.63
    pressure   int64 8B ...
Data variables:
    windspeed  (time, lat, lon) float64 4GB ...
Attributes:
    long_name:      Air Temperature
    standard_name:  air_temperature
    units:          K
    cell_methods:   pressure: point time: point (interval: 6H) area: interpol...
    grid_mapping:   crs

In [7]:
%%time
# testing
VARU = "wnd_ucmp"
VARV = "wnd_vcmp"

start_year = 1990
end_year = 2019

regridder=None
# target grid for regridding
ds_target = xr.open_dataset("/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_ACCESS-CM2_historical_r4i1p1f1_BOM_BARPA-R_v1-r1_6hr_1979.nc")

def _preprocess(ds):
    try:
        ds = ds.drop_vars( [ "forecast_period", "forecast_reference_time", "latitude_longitude"],)\
            .sel({"pressure":250}, method = "nearest")\
            .sel(latitude= slice(-53.58 , 13.63), longitude=slice(88.04 , 207.9))
    except:
        ds = ds.drop_vars( ["latitude_longitude"],)\
            .sel({"pressure":250}, method = "nearest")\
            .sel(latitude= slice(-53.58 , 13.63), longitude=slice(88.04 , 207.9))
    return ds     

for year in np.arange(start_year, end_year+1):
    new_file = f"/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_{year}.nc"
    if os.path.exists(new_file):
        print(f"File '{new_file}' already exists.")
        continue
        
    # u component of wind
    print("filelist u...")
    filelist = glob.glob(f"/g/data/cj37/BARRA/BARRA_R/v1/analysis/prs/{VARU}/{year}/*/{VARU}-an-prs-PT0H-BARRA_R-v1-*T*00Z.sub.nc")
    filelist.sort()

    print("open_mfdataset u...")
    ds_u = xr.open_mfdataset(filelist, preprocess=_preprocess, combine="nested", concat_dim="time", parallel=True,)\
            .chunk({"time":-1, "latitude":220, "longitude":112})\
            .rename({f"{VARU}":"ua250", "latitude":"lat", "longitude":"lon"})

    # v component of wind
    print("filelist v...")
    filelist = glob.glob(f"/g/data/cj37/BARRA/BARRA_R/v1/analysis/prs/{VARV}/{year}/*/{VARV}-an-prs-PT0H-BARRA_R-v1-*T*00Z.sub.nc")
    filelist.sort()

    print("open_mfdataset v...")
    ds_v = xr.open_mfdataset(filelist, preprocess=_preprocess, combine="nested", concat_dim="time", parallel=True,)\
            .chunk({"time":-1, "latitude":220, "longitude":112})\
            .rename({f"{VARV}":"va250", "latitude":"lat", "longitude":"lon"})

    # calculate windspeed (lazy)
    print("calculate windspeed")
    ds_windspeed = xr.Dataset({"windspeed":xr.ufuncs.hypot(ds_u["ua250"], ds_v["va250"])}
                             ).chunk({"time":120, "lat":-1, "lon":-1})

    ds_u.close()
    ds_v.close()
    
    if regridder is None:
        print("calculate regridder...")
        regridder = xe.Regridder(ds_windspeed, ds_target, "bilinear")
    print(f"regrid and compute {year}...")
    ds_regridded = regridder(ds_windspeed).compute()
    try:
        ds_regridded.to_netcdf(new_file)
        print(f"Saved {new_file}")
    except:
        print(f"cannot save {new_file}")
    ds_windspeed.close()
    ds_regridded.close()
    

File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1990.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1991.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1992.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1993.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1994.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1995.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_1996.nc' already exists.
File '/scratch/v46/gt3409/TMP_windspeed/TMP_wind

HDF5-DIAG: Error detected in HDF5 (1.14.3) thread 0:
  #000: H5A.c line 1891 in H5Aiterate2(): invalid location identifier
    major: Invalid arguments to routine
    minor: Inappropriate type
  #001: H5VLint.c line 1741 in H5VL_vol_object(): invalid identifier type to function
    major: Invalid arguments to routine
    minor: Inappropriate type
[gadi-cpu-bdw-0027:3077802:0:3077958] Caught signal 11 (Segmentation fault: Sent by the kernel at address (nil))
==== backtrace (tid:3077958) ====
 0  /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/h5py/../../../././libucs.so.0(ucs_handle_error+0x2fd) [0x153dac5ddc1d]
 1  /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/h5py/../../../././libucs.so.0(+0x2de11) [0x153dac5dde11]
 2  /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/h5py/../../../././libucs.so.0(+0x2dfda) [0x153dac5ddfda]
 3  /lib64/libpthread.so.0(+0x12990) [0x153df19b7990

calculate windspeed
calculate regridder...


[gadi-cpu-bdw-0027.gadi.nci.org.au:3076940] shmem: mmap: an error occurred while determining whether or not /jobfs/150578280.gadi-pbs/ompi.gadi-cpu-bdw-0027.12872/jf.0/2149253120/shared_mem_cuda_pool.gadi-cpu-bdw-0027 could be created.
[gadi-cpu-bdw-0027.gadi.nci.org.au:3076940] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


regrid and compute 2017...
Saved /scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_2017.nc
filelist u...
open_mfdataset u...
filelist v...
open_mfdataset v...
calculate windspeed
regrid and compute 2018...
Saved /scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_2018.nc
filelist u...
open_mfdataset u...
filelist v...
open_mfdataset v...


[gadi-cpu-bdw-0027:3077823:0:3077960] Caught signal 11 (Segmentation fault: address not mapped to object at address 0x78)
==== backtrace (tid:3077960) ====
 0  /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/h5py/../../../././libucs.so.0(ucs_handle_error+0x2fd) [0x146f0154bc1d]
 1  /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/h5py/../../../././libucs.so.0(+0x2de11) [0x146f0154be11]
 2  /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/h5py/../../../././libucs.so.0(+0x2dfda) [0x146f0154bfda]
 3  /lib64/libpthread.so.0(+0x12990) [0x146f48aa4990]
 4  /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/netCDF4/../../../libnetcdf.so.19(nc4_find_nc_grp_h5+0x6b) [0x146ef0c8f00b]
 5  /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/netCDF4/../../../libnetcdf.so.19(NC4_inq_format+0x2a) [0x146ef0c9842a]
 6  /g/data/hh

calculate windspeed
regrid and compute 2019...
Saved /scratch/v46/gt3409/TMP_windspeed/TMP_windspeed_AUS-15_BARRA-R_evaluation_r1i1p1f1_BOM_BARPA-R_v1-r1_6hr_2019.nc
CPU times: user 10min 32s, sys: 1min 32s, total: 12min 5s
Wall time: 13min 56s
